# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [22]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [23]:
MYTABLE='katepearson' + '_data'

In [24]:
host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

In [25]:
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [26]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [27]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'

### SOLUTION GOES HERE
df = pd.read_sql_query("SHOW TABLES", conn)

katepearson_data = {}
for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    katepearson_data[name] = pd.read_sql("SELECT * FROM "+name,conn)

Medicaid_EP_Hospital_Type
NPI
Population
Provider_Name
Test
a85531647_data
aavila_data
aavila_data_NPI
aavila_data_new
aavila_data_year
aavila_datax
abbieslu
abbieslu_NPI
abbieslu_ipv
abbieslu_npi
abbieslu_year
achenliam
achenliam_data
achenliam_ipv
apuryear_data
arefehesmaeilpour_data
ashley_data
btabman_data
corona_counts
county_population
dhumphrey420_data
ed_visits
hospital_services
hospitals
jdgendron_data
jmccar23_data
kaylaschmidt_data
kenny0415_data
khalafh_data
kodurus_data
lchen_data_ipv
liannecota_data
liannecota_newdata
paulboal_data
petersluu_data
population
ppp_data
qichen_data
sam
sampele
sneha-koduru_data


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '-koduru_data' at line 1")
[SQL: SELECT * FROM sneha-koduru_data]
(Background on this error at: http://sqlalche.me/e/f405)

In [20]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'hds5210.katepearson_data' doesn't exist")
[SQL: SELECT * FROM katepearson_data]
(Background on this error at: http://sqlalche.me/e/f405)

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [ ]:
tables = katepearson_data['hospitals'].merge(population['Zip_Code', 'popuation'])
max_pop = MAX('population')

answer = ['a', 'list', 'of', 'NPI', 'like', '1194016923']

In [ ]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [ ]:
### SOLUTION GOES HERE
answer = pd.read_sql_query("""
  SELECT
  COUNT(NPI),
  Medicaid_EP_Hospital_Type, 
  Zip_Code, 
  SUM(Total_payments),
  SUM(Total_payments) / COUNT(NPI) AS Avg_Pay_per_Capita
  FROM
  tables
  GROUP BY
  Medicaid_EP_Hospital_Type, 
  Zip_Code
    
""", conn)

In [ ]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
